### **Complex model using full quantum**

In [ ]:
!pip install datasets transformers optuna safetensors

import torch
import torch.nn as nn
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import time

class AdvancedQuantumInspiredModel(nn.Module):
    def __init__(self, roberta_model):
        super(AdvancedQuantumInspiredModel, self).__init__()
        self.roberta_model = roberta_model
        self.conv1 = nn.Conv1d(768, 768, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(768, 768, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(768 * 128, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta_model(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.hidden_states[-1].transpose(1, 2)

        def process_path1(x):
            return F.relu(self.conv1(x))

        def process_path2(x):
            return F.relu(self.conv2(x))

        with ThreadPoolExecutor(max_workers=2) as executor:
            futures = [executor.submit(process_path1, x), executor.submit(process_path2, x)]
            results = [f.result() for f in futures]

        x = (results[0] + results[1]) / 2

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))

        if torch.rand(1).item() > 0.5:
            x = x * torch.rand_like(x)

        logits = self.fc2(x)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.fc2.out_features), labels.view(-1))
            return loss, logits
        else:
            return logits

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
roberta_model = AutoModelForSequenceClassification.from_pretrained('roberta-base', output_hidden_states=True)
quantum_model = AdvancedQuantumInspiredModel(roberta_model)

# Load dataset
dataset = load_dataset('super_glue', 'boolq')
metric = load_metric('super_glue', 'boolq')

def preprocess_function(examples):
    return tokenizer(examples['question'], examples['passage'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

train_dataset = encoded_dataset['train'].select(range(50))
eval_dataset = encoded_dataset['validation'].select(range(50))

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

class AdvancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        outputs = model(inputs['input_ids'], inputs['attention_mask'], labels=labels)
        loss = outputs[0]
        return (loss, outputs) if return_outputs else loss

trainer = AdvancedTrainer(
    model=quantum_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: metric.compute(predictions=torch.argmax(torch.tensor(p.predictions), axis=1), references=p.label_ids),
)

print("Training and evaluating Quantum-Inspired model...")
trainer.model = quantum_model
trainer.train()

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=128)

quantum_model.eval()
start_time = time.time()
with torch.no_grad():
    outputs = quantum_model(inputs['input_ids'], inputs['attention_mask'])
end_time = time.time()
quantum_inference_time = end_time - start_time
print(f"Quantum-Inspired Model Inference Time: {quantum_inference_time} seconds")

print("Evaluating the quantum-inspired model...")
trainer.model = quantum_model
results = trainer.evaluate()

print("Model results:", results)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training and evaluating Quantum-Inspired model...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.649302,0.780000


Quantum-Inspired Model Inference Time: 0.5571029186248779 seconds
Evaluating the quantum-inspired model...


Model results: {'eval_loss': 0.6560001969337463, 'eval_accuracy': 0.76, 'eval_runtime': 24.9094, 'eval_samples_per_second': 2.007, 'eval_steps_per_second': 0.281, 'epoch': 1.0}


In [ ]:
!pip install transformers datasets

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import time

tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base')

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=128)

model.eval()
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_time = time.time()
baseline_inference_time = end_time - start_time

print(f"Baseline RoBERTa Model Inference Time: {baseline_inference_time} seconds")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Baseline RoBERTa Model Inference Time: 0.8747677803039551 seconds
Baseline RoBERTa Model Accuracy: 0
